In [12]:
import pandas as pd
import numpy as np
from functools import reduce
civicdata = pd.read_excel('data/mumbai/civic_data_ Jun17-July17.xlsx' , encoding='utf-8')
# Geo code and geo level
geography = pd.read_csv('data/geography.csv',encoding='UTF-8')
geo = geography[['geo_level','geo_code','name']]

# Calculating country data
def setCountryData(df,cols):
    for x in cols:
         df.loc[df['geo_level'] == 'country', x]= df.loc[df['geo_level'] == 'state', x].sum()
    return df;


# Calculating geographic data
def getGeoData(df):
    cols = list(df.columns.values)[3:]
    df = df.replace(',','',regex=True)
    df[cols] = df[cols].astype(float)
    df = df.rename(index=str, columns={'STATE NAME': 'name'})
    dfState = df.groupby(['name'])[cols].sum().reset_index()
   
    dfDis   = df.groupby(['DISTRICT NAME'])[cols].sum().reset_index()
    dfDis   = dfDis.rename(index=str, columns={'DISTRICT NAME': 'name'})
    dfDis['name'] = dfDis['name'].str.title()
    
    dfCom = pd.concat([dfState, dfDis])
    dfgeo = geo.merge(dfCom ,on=['name'],how='left')
    dfgeo = dfgeo.sort_values(by=['geo_level','name'])
    dfgeo = dfgeo.fillna(0)
    dfgeoCountry = setCountryData(dfgeo,cols)
    return dfgeoCountry

# Calculating census data
def getCensusData(df,varname):
    dfgeo = getGeoData(df)
    dfgeoFinal = pd.melt(dfgeo, id_vars=['geo_level', 'geo_code','name'], var_name=varname, value_name="total")
    dfgeoFinal = dfgeoFinal.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1)
    dfgeoFinal['total'] = dfgeoFinal['total'].astype(int)
    return  dfgeoFinal

# Calculating school data
def getSchooldata(schooldf):
    schooldf.columns = schooldf.columns.str.split('(').str[0]
    schooldf.columns = schooldf.columns.str.strip()
    Schooldfgeo      = getGeoData(schooldf)
    return Schooldfgeo

In [13]:
civicdata['STATE NAME'] = 'MAHARASHTRA'
civicdata['DISTRICT NAME'] = 'Mumbai'
civicdata  = civicdata.rename(index=str, columns={'Complaint Type':'civic_complaint','Status of Complaint':'Complaint_status','Days Elasped':'days_elapsed'})
civicdatadf = civicdata[['STATE NAME','DISTRICT NAME','civic_complaint','days_elapsed']]
civicdatadf.head()

,STATE NAME,DISTRICT NAME,civic_complaint,days_elapsed
0,MAHARASHTRA,Mumbai,Replacement of Missing/ Damaged Manhole,100
1,MAHARASHTRA,Mumbai,Others (Estate),82
2,MAHARASHTRA,Mumbai,Air Pollution,68
3,MAHARASHTRA,Mumbai,Resurfacing of Road,6
4,MAHARASHTRA,Mumbai,Bad Patches/ Potholes on the Roads,58


# Making pivot table

In [14]:
civicdatadff      = civicdatadf.rename(index=str, columns={'STATE NAME': 'name'})
civicdataPivot   = civicdatadff.pivot_table(values="days_elapsed", index=['name','DISTRICT NAME'], columns='civic_complaint')
civicdataPivotDF = civicdataPivot.reset_index()
civicdataPivotDF
cols1 = list(civicdataPivotDF.columns.values)[2:]
dfState      = civicdataPivotDF.groupby(['name'])[cols1].sum().reset_index()
dfDis        = civicdataPivotDF.groupby(['DISTRICT NAME'])[cols1].sum().reset_index()
dfDis        = dfDis.rename(index=str, columns={'DISTRICT NAME': 'name'})
dfCom        = pd.concat([dfState, dfDis])
dfgeo        = geo.merge(dfCom ,on=['name'],how='left')
dfgeo
dffinalcivic = pd.melt(dfgeo, id_vars=['geo_level', 'geo_code','name'], var_name='civic_complaint', value_name="total")
dffinalcivic  = dffinalcivic.fillna(0)
dffinalcivicf = dffinalcivic.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1)
dffinalcivicf['total'] = dffinalcivicf['total'].astype(int)
dffinalcivicf.insert(2,'geo_version','2011')
dffinalcivicf.head()
#dffinalcivicf.to_csv('data/civiccomplaint.csv', index=False)

,geo_level,geo_code,geo_version,civic_complaint,total
0,country,IN,2011,Air Pollution,0
1,country,IN,2011,Bad Patches/ Potholes on the Roads,0
2,country,IN,2011,Bad Quality of material given to student,0
3,country,IN,2011,Buildings Related,0
4,country,IN,2011,Burst water main lines,0


# Top 5 time taken to resolve civic complaint

In [15]:
civictemp = civicdatadf.groupby(['STATE NAME','DISTRICT NAME','civic_complaint'])['days_elapsed'].mean().reset_index()
civictopten = civictemp.nlargest(5, columns=['days_elapsed'])
civictoptendf   = civictopten.rename(index=str, columns={'STATE NAME': 'name'})
civictoptenPivot   = civictoptendf.pivot_table(values="days_elapsed", index=['name','DISTRICT NAME'], columns='civic_complaint')
civictoptenPivotDF   = civictoptenPivot.reset_index()

cols = list(civictoptenPivotDF.columns.values)[2:]
dfState      = civictoptenPivotDF.groupby(['name'])[cols].sum().reset_index()
dfDis        = civictoptenPivotDF.groupby(['DISTRICT NAME'])[cols].sum().reset_index()
dfDis        = dfDis.rename(index=str, columns={'DISTRICT NAME': 'name'})
dfCom        = pd.concat([dfState, dfDis])
dfgeo        = geo.merge(dfCom ,on=['name'],how='left')
dfgeo

dffinalcivictopten = pd.melt(dfgeo, id_vars=['geo_level', 'geo_code','name'], var_name='civic_complaint', value_name="total")
dffinalcivictopten  = dffinalcivictopten.fillna(0)
dffinalcivictopten = dffinalcivictopten.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1)
dffinalcivictopten['total'] = dffinalcivictopten['total'].astype(int)
dffinalcivictopten.insert(2,'geo_version','2011')
dffinalcivictopten.insert(4,'year','2017')
dffinalcivictopten.head()
#dffinalcivictopten.to_csv('data/civiccomplaint_mum_2017.csv', index=False)

,geo_level,geo_code,geo_version,civic_complaint,year,total
0,country,IN,2011,Bad Quality of material given to student,2017,0
1,country,IN,2011,No warning/ alarm system,2017,0
2,country,IN,2011,Pertaining to rent,2017,0
3,country,IN,2011,Unauthorised addition/ alteration in the premi...,2017,0
4,country,IN,2011,Unauthorised shed on building in premise,2017,0


# Reading hospital data

In [16]:
ColabaMunicipal_Dispensary = pd.read_excel('data/mumbai/ColabaMunicipal Dispensary_PW_GP_GP_RG.xlsx' , encoding='utf-8')
HeadOffice_Dispensary = pd.read_excel('data/mumbai/HeadOffice(H.O)Dispensary_KG_GP_GP_RG.xlsx' , encoding='utf-8')
MarutiLane_Dispensary = pd.read_excel('data/mumbai/MarutiLaneDispensary_KG_GP_GP_RG.xlsx' , encoding='utf-8')
SabooSiddhique_Dispensary = pd.read_excel('data/mumbai/SabooSiddhiqueRoadDispensary_KG_PW_GP_GP_RG.xlsx' , encoding='utf-8')
ShahidBhagatSingh_Dispensary = pd.read_excel('data/mumbai/ShahidBhagatSinghRoadDispensary_KG_GP_GP_RG.xlsx' , encoding='utf-8')
ShahidBhagatSingh_Dispensary.head()

,Sr.No.,Disease,Occurrence,Reported Date,Comments,Verification Status
0,1,Tuberculosis,1,2016-09-01,NaN,Yes
1,2,Tuberculosis,1,2016-11-01,NaN,Yes
2,3,Diarrhoea,16,2016-08-01,NaN,Yes
3,4,Diarrhoea,5,2016-09-01,NaN,Yes
4,5,Diarrhoea,5,2016-10-01,NaN,Yes


# Appending multiple dispensary

In [17]:
Appended_dispensary = ColabaMunicipal_Dispensary.append([HeadOffice_Dispensary, MarutiLane_Dispensary,SabooSiddhique_Dispensary,ShahidBhagatSingh_Dispensary ])
Appended_dispensary.insert(1,'STATE NAME','MAHARASHTRA')
Appended_dispensary.insert(2,'DISTRICT NAME','Mumbai')

Appended_dispensaryDF = Appended_dispensary[['STATE NAME','DISTRICT NAME','Disease','Occurrence']]
Appended_dispensaryfinalDF = Appended_dispensaryDF.groupby(['STATE NAME','DISTRICT NAME','Disease'])['STATE NAME','DISTRICT NAME','Disease','Occurrence'].sum().reset_index()
Appended_dispensaryfinalDF.head()

,STATE NAME,DISTRICT NAME,Disease,Occurrence
0,MAHARASHTRA,Mumbai,Abdomen Pain,146
1,MAHARASHTRA,Mumbai,Acid Peptic Disease (APD),181
2,MAHARASHTRA,Mumbai,Acute Gastro Entritis (AGE),3
3,MAHARASHTRA,Mumbai,Acute Respiratory Infection,10
4,MAHARASHTRA,Mumbai,Anaemia,114


# Top 5 diseases occurence in Mumbai

In [21]:
Appended_dispensarytop5    = Appended_dispensaryfinalDF.nlargest(5, columns=['Occurrence'])

Appended_dispensarytop5    = Appended_dispensarytop5 .rename(index=str, columns={'STATE NAME': 'name'})
Appended_dispensaryPivot   = Appended_dispensarytop5.pivot_table(values="Occurrence", index=['name','DISTRICT NAME'], columns='Disease')
Appended_dispensarydf      = Appended_dispensaryPivot.reset_index()


cols1 = list(Appended_dispensarydf.columns.values)[2:]
dfState      = Appended_dispensarydf.groupby(['name'])[cols1].sum().reset_index()
dfDis        = Appended_dispensarydf.groupby(['DISTRICT NAME'])[cols1].sum().reset_index()
dfDis        = dfDis.rename(index=str, columns={'DISTRICT NAME': 'name'})
dfCom        = pd.concat([dfState, dfDis])
dfgeo        = geo.merge(dfCom ,on=['name'],how='left')
dfgeo
dffinaldispensary = pd.melt(dfgeo, id_vars=['geo_level', 'geo_code','name'], var_name='disease', value_name="total")
dffinaldispensary  = dffinaldispensary.fillna(0)
dffinaldispensaryf = dffinaldispensary.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1)
dffinaldispensaryf['total'] = dffinaldispensaryf['total'].astype(int)
dffinaldispensaryf.insert(2,'geo_version','2011')
dffinaldispensaryf.insert(4,'year','2016')
dffinaldispensaryf.head()
#dffinaldispensaryf.to_csv('data/test_mumbai.csv.gz', index=False, compression='gzip')

,geo_level,geo_code,geo_version,disease,year,total
0,country,IN,2011,Common Cold,2016,0
1,country,IN,2011,Influenza Like Illness (I.L.I.),2016,0
2,country,IN,2011,Injury,2016,0
3,country,IN,2011,Joint Pain/ Arthiritis,2016,0
4,country,IN,2011,Upper Respectory Track Infection (URTI),2016,0


In [11]:
from IPython.display import HTML
import base64


def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(dffinaldispensaryf,'xyz','temp.csv')